## Hybrid turning approach

In [10]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
from gymnasium.utils.env_checker import check_env
from IPython.display import Video
from enum import Enum, auto

from flygym.mujoco import Parameters
from flygym.mujoco.arena import FlatTerrain
from flygym.mujoco.examples.obstacle_arena import ObstacleOdorArena
from flygym.mujoco.examples.turning_controller import HybridTurningNMF

## Setting up the simulation

In [11]:
import flygym.mujoco
from tqdm import trange

# We start by creating a simple arena
flat_terrain_arena = FlatTerrain()

# Then, we add visual and olfactory features on top of it
arena = ObstacleOdorArena(
    terrain=flat_terrain_arena,
    obstacle_positions=np.array([(7.5, 0), (12.5, 5), (17.5, -5)]),
    marker_size=0.5,
    obstacle_colors=[(0.14, 0.14, 0.2, 1), (0.2, 0.8, 0.2, 1), (0.2, 0.2, 0.8, 1)],
    user_camera_settings=((13, -18, 9), (np.deg2rad(65), 0, 0), 45),
)

contact_sensor_placements = [
    f"{leg}{segment}"
    for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]

run_time = 1
sim_params = flygym.mujoco.Parameters(
    timestep=1e-4, 
    render_mode="saved", 
    render_playspeed=0.1, 
    draw_contacts=False,
    render_camera="user_cam"
)

nmf = HybridTurningNMF(
    sim_params=sim_params,
    init_pose="stretch",
    spawn_pos=(13, -5, 0.2),
    spawn_orientation=(0, 0, np.pi / 2 + np.deg2rad(70)),
    contact_sensor_placements=contact_sensor_placements,
    arena=arena
)


## Running simulation

## Running simulation with turning


In [17]:
from enum import Enum, auto

# random state seed for reproducibility
seed = 1

class State(Enum):
    GO_STRAIGHT = 1
    TURN_LEFT = 2
    TURN_RIGHT = 3
    REVERSE_LEFT = 4
    REVERSE_RIGHT = 5
    REACH_AND_TURN = 6

# Example of setting a current state
current_state = State.GO_STRAIGHT

decision_interval = 0.2
run_time = 2
num_decision_steps = int(run_time / decision_interval)
physics_steps_per_decision_step = int(decision_interval / sim_params.timestep)

low_force_thresh = 1
high_force_thresh = 3
enforce_time = 0
delay = 0.2

obs_hist = []
odor_history = []
obs, _ = nmf.reset(seed)
bias = np.array([0,0])
for i in trange(int(run_time / nmf.sim_params.timestep)):
    curr_time = i * nmf.sim_params.timestep
    left_sense = np.array(obs["contact_forces"][:5, 0:2])
    right_sense = np.array(obs["contact_forces"][18:23, 0:2])
    left_sense_sum = left_sense.sum()
    right_sense_sum = right_sense.sum()

    if right_sense_sum > low_force_thresh:
        if current_state == State.GO_STRAIGHT:
            if right_sense_sum > high_force_thresh:
                current_state = State.REVERSE_LEFT
                # reverse while turning left function
            else:
                current_state = State.TURN_LEFT
                bias = np.array([-2, 0])
        elif current_state == State.TURN_LEFT:
            current_state = State.REVERSE_LEFT
            # reverse while turning left function
        elif current_state == State.REVERSE_LEFT:
            # reverse while turning left function
            trash = 0
        elif current_state == State.TURN_RIGHT or current_state == State.REVERSE_RIGHT:
            current_state == State.REACH_AND_TURN
            # call reach and turn function
        enforce_time =  curr_time + delay
    elif left_sense_sum > low_force_thresh:
        if current_state == State.GO_STRAIGHT:
            if left_sense_sum > high_force_thresh:
                current_state = State.REVERSE_RIGHT
                # reverse while turning right function
            else:
                current_state = State.TURN_RIGHT
                bias = np.array([0, -2])
        elif current_state == State.TURN_LEFT or current_state == State.REVERSE_LEFT:
            current_state == State.REACH_AND_TURN
            # call reach and turn function
        elif current_state == State.TURN_RIGHT:
            current_state = State.REVERSE_RIGHT
            # reverse while turning right function
        elif current_state == State.REACH_AND_TURN:
            # reverse while turning left function
            trash = 0
        enforce_time =  curr_time + delay
    elif curr_time >= enforce_time:
        current_state = State.GO_STRAIGHT
        bias = np.array([0,0])
    
    control_signal = np.array([1, 1]) + bias

    obs, reward, terminated, truncated, info = nmf.step(control_signal)
    nmf.render()

nmf.save_video("./outputs/pillars.mp4")
Video("./outputs/pillars.mp4")

100%|██████████| 20000/20000 [02:13<00:00, 150.34it/s]
